In [2]:
import torch
import numpy
import torch.nn.functional as F
from torch import nn

In [32]:
batchsize=2
max_num_src_words=8
max_num_tgt_words=8
model_dim=8

max_src_seq_len=5
max_tgt_seq_len=5
max_position_len=5
# src_len= torch.randint(2,5,(batchsize,))
# tgt_len= torch.randint(2,5,(batchsize,))
src_len=torch.Tensor([2,4]).to(torch.int32)
tgt_len=torch.Tensor([4,3]).to(torch.int32)
#单词索引序列构成源句子和目标锯子，并且做了padding
src_seq=torch.cat([torch.unsqueeze(F.pad(torch.randint(1,max_num_src_words,(L,)),(0,max_src_seq_len-L)),0) for L in src_len])
tgt_seq=torch.cat([torch.unsqueeze(F.pad(torch.randint(1,max_num_tgt_words,(L,)),(0,max_tgt_seq_len-L)),0) for L in tgt_len])
print(src_seq)
print(tgt_seq)

tensor([[3, 4, 0, 0, 0],
        [5, 6, 5, 7, 0]])
tensor([[2, 7, 3, 5, 0],
        [3, 3, 7, 0, 0]])


In [20]:
#构造embedding(索引table构造)
src_embedding_table=nn.Embedding(max_num_src_words+1,model_dim)
tgt_embedding_table=nn.Embedding(max_num_tgt_words+1,model_dim)
src_embedding=src_embedding_table(src_seq)
tgt_embedding=tgt_embedding_table(tgt_seq)
print(src_embedding)
print(tgt_embedding)

tensor([[[ 0.2328, -0.0056,  0.2100, -1.1865, -0.9385,  0.4304, -0.9666,
          -0.5284],
         [ 1.2238,  1.2813,  0.2205, -0.6406, -0.4276, -0.4490, -0.5950,
          -0.4858],
         [ 0.1921, -1.1331, -0.2075, -0.0905, -0.3573, -0.9686, -0.9755,
          -1.3524],
         [ 0.1921, -1.1331, -0.2075, -0.0905, -0.3573, -0.9686, -0.9755,
          -1.3524],
         [ 0.1921, -1.1331, -0.2075, -0.0905, -0.3573, -0.9686, -0.9755,
          -1.3524]],

        [[ 0.2328, -0.0056,  0.2100, -1.1865, -0.9385,  0.4304, -0.9666,
          -0.5284],
         [-1.2628,  0.6543, -0.9529, -1.5322,  0.0348,  1.7021, -0.3497,
          -0.5548],
         [ 1.2238,  1.2813,  0.2205, -0.6406, -0.4276, -0.4490, -0.5950,
          -0.4858],
         [-0.1172,  0.9872,  0.5635,  0.5389, -0.1481,  0.3537,  1.5191,
          -0.8734],
         [ 0.1921, -1.1331, -0.2075, -0.0905, -0.3573, -0.9686, -0.9755,
          -1.3524]]], grad_fn=<EmbeddingBackward0>)
tensor([[[-0.3116, -0.3202, -0.3221,

In [40]:
#构造position embedding
position_mat=torch.arange(max_position_len).reshape(-1,1)
i_mat=torch.pow(10000,torch.arange(0,8,2).reshape(1,-1)/model_dim)
pe_embedding_table=torch.zeros(max_position_len,model_dim)
pe_embedding_table[:,0::2]=torch.sin(position_mat/i_mat)
pe_embedding_table[:,1::2]=torch.cos(position_mat/i_mat)

pe_embedding=nn.Embedding(max_position_len,model_dim)
pe_embedding.weight=nn.Parameter(pe_embedding_table,requires_grad=False)

src_pos=torch.cat([torch.unsqueeze(torch.arange(max(src_len)),0) for _ in src_len]).to(torch.int32)
tgt_pos=torch.cat([torch.unsqueeze(torch.arange(max(tgt_len)),0) for _ in tgt_len]).to(torch.int32)

src_pe_embedding=pe_embedding(src_pos)
tgt_pe_embedding=pe_embedding(tgt_pos)
print( src_pe_embedding)
print( tgt_pe_embedding)

tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           1.0000e+00,  0.0000e+00,  1.0000e+00],
         [ 8.4147e-01,  5.4030e-01,  9.9833e-02,  9.9500e-01,  9.9998e-03,
           9.9995e-01,  1.0000e-03,  1.0000e+00],
         [ 9.0930e-01, -4.1615e-01,  1.9867e-01,  9.8007e-01,  1.9999e-02,
           9.9980e-01,  2.0000e-03,  1.0000e+00],
         [ 1.4112e-01, -9.8999e-01,  2.9552e-01,  9.5534e-01,  2.9995e-02,
           9.9955e-01,  3.0000e-03,  1.0000e+00]],

        [[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           1.0000e+00,  0.0000e+00,  1.0000e+00],
         [ 8.4147e-01,  5.4030e-01,  9.9833e-02,  9.9500e-01,  9.9998e-03,
           9.9995e-01,  1.0000e-03,  1.0000e+00],
         [ 9.0930e-01, -4.1615e-01,  1.9867e-01,  9.8007e-01,  1.9999e-02,
           9.9980e-01,  2.0000e-03,  1.0000e+00],
         [ 1.4112e-01, -9.8999e-01,  2.9552e-01,  9.5534e-01,  2.9995e-02,
           9.9955e-01,  3.0000e-03,  1.0000e+00]

In [ ]:
valid_encoder_pos = torch.unsqueeze(torch.cat([
    torch.unsqueeze(F.pad(torch.ones(L), (0, max_src_seq_len - L)), 0) 
    for L in src_len
]), 2)

valid_encoder_pos_matrix = torch.bmm(valid_encoder_pos, valid_encoder_pos.transpose(1, 2))
invalid_encoder_pos_matrix = 1 - valid_encoder_pos_matrix
mask_encoder_self_attention = invalid_encoder_pos_matrix.to(torch.bool)

score = torch.randn(batch_size, max_src_seq_len, max_src_seq_len)

masked_score = score.masked_fill(mask_encoder_self_attention, -1e9)
prob = F.softmax(masked_score, -1)

print("mask形状:", mask_encoder_self_attention.shape)
print("score形状:", score.shape)
print("masked_score:", masked_score)

mask形状: torch.Size([2, 5, 5])
score形状: torch.Size([2, 5, 5])
masked_score: tensor([[[-8.0664e-01,  3.4761e+00, -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [ 1.9000e+00,  9.4254e-02, -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09]],

        [[ 1.5015e+00,  7.2953e-01,  8.0450e-01, -1.1050e+00, -1.0000e+09],
         [-1.4932e+00, -6.4297e-02,  1.8460e+00, -5.7211e-01, -1.0000e+09],
         [-3.0728e-01,  6.9106e-01,  7.9332e-01, -1.2655e+00, -1.0000e+09],
         [ 9.9577e-01,  1.0239e+00,  1.3199e+00,  4.8249e-01, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09]]])
